In [19]:
from optibook.synchronous_client import Exchange
import time
from optibook.synchronous_client import Exchange


import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


In [ ]:
e = Exchange()
a = e.connect()

instrument_id = 'PHILIPS_A'
instrument_id2 = 'PHILIPS_B'

In [ ]:
# ////////////////// timer //////////////////
start = time.time() # added by qdou

for i in range(10000): 
    time.sleep(0.01)

    book = e.get_last_price_book(instrument_id)
    book2 = e.get_last_price_book(instrument_id2)

    try:
        A_Bid_B_Ask_Spread = book.bids[0].price-book2.asks[0].price
        if A_Bid_B_Ask_Spread > 0:
            print(A_Bid_B_Ask_Spread, '/A Bid - B Ask')
        B_Bid_A_Ask_Spread = book2.bids[0].price-book.asks[0].price
        if B_Bid_A_Ask_Spread > 0:
            print(B_Bid_A_Ask_Spread, '/B Bid - A Ask')
    except IndexError:
        pass
    
    try:    
        for s, p in e.get_positions().items():
            if p > 100:
                e.insert_order(s, price=1, volume=p, side='ask', order_type='ioc')
                print('clear inventory of ', p)
                time.sleep(5)
            elif p < -100:
                e.insert_order(s, price=100000, volume=-p, side='bid', order_type='ioc')
                print('clear inventory of ', p)
                time.sleep(5)
     
    except IndexError:
        pass
    

    positions = e.get_positions() # added by qdou
    if A_Bid_B_Ask_Spread > 0:
        print('A bid',book.bids[0].price,'B ask',book2.asks[0].price)
        ArbiVol = min(book.bids[0].volume, book2.asks[0].volume, 100, 250-positions["PHILIPS_A"], 250-positions["PHILIPS_B"],250+positions["PHILIPS_A"],250+positions["PHILIPS_B"])
        print('A_Bid_B_Ask vol = ',ArbiVol)
        # begining with the side with less liquidity > lower volume
        if book2.asks[0].volume > book.bids[0].volume:
            sell = e.insert_order(instrument_id, price=book.bids[0].price, volume=ArbiVol, side='ask', order_type='limit')
            print('sell A at ',book.bids[0].price)
            buy = e.insert_order(instrument_id2, price=book2.asks[0].price, volume=ArbiVol, side='bid', order_type='limit')
            print('buy B at ',book2.asks[0].price)
            print(f"Order Id: {buy}")
            print(f"Order Id: {sell}")
            print(ArbiVol,'/A Bid - B Ask')
        else:
            buy = e.insert_order(instrument_id2, price=book2.asks[0].price, volume=ArbiVol, side='bid', order_type='limit')
            print('buy B at ',book2.asks[0].price)
            sell = e.insert_order(instrument_id, price=book.bids[0].price, volume=ArbiVol, side='ask', order_type='limit')
            print('sell A at ',book.bids[0].price)
            print(f"Order Id: {buy}")
            print(f"Order Id: {sell}")
            print(ArbiVol,'/A Bid - B Ask')

    elif B_Bid_A_Ask_Spread > 0:
        print('B bid' ,book2.bids[0].price, 'A ask' ,book.asks[0].price)
        ArbiVol = min(book2.bids[0].volume, book.asks[0].volume, 100,250-positions["PHILIPS_A"], 250-positions["PHILIPS_B"],250+positions["PHILIPS_A"],250+positions["PHILIPS_B"])
        print('B_Bid_A_Ask vol = ',ArbiVol)
        # begining with the side with less liquidity > lower volume
        if book.asks[0].volume > book2.bids[0].volume:
            sell = e.insert_order(instrument_id2, price=book2.bids[0].price, volume=ArbiVol, side='ask', order_type='limit')
            print('sell B at ',book2.bids[0].price)
            buy = e.insert_order(instrument_id, price=book.asks[0].price, volume=ArbiVol, side='bid', order_type='limit')
            print('buy A at ',book.asks[0].price)
            print(f"Order Id: {buy}")
            print(f"Order Id: {sell}")
            print(ArbiVol,'/B Bid - A Ask')

        else:
            buy = e.insert_order(instrument_id, price=book.asks[0].price, volume=ArbiVol, side='bid', order_type='limit')
            print('buy A at ',book.asks[0].price)
            sell = e.insert_order(instrument_id2, price=book2.bids[0].price, volume=ArbiVol, side='ask', order_type='limit')
            print('sell B at ',book2.bids[0].price)
            print(f"Order Id: {buy}")
            print(f"Order Id: {sell}")
            print(ArbiVol,'/ A AskB - Bid ')
            

    end = time.time()
    #print('The time elapsed between connection and now (after execution) is {}'.format(end - start))
    # ////////////////// timer //////////////////

In [ ]:
# Check current positions and PnL
positions = e.get_positions()
for p in positions:
    print(p, positions[p])
    
pnl = e.get_pnl()
print('PnL =',pnl)

In [179]:
e.get_positions().items()

dict_items([('PHILIPS_A', 187), ('PHILIPS_B', -218)])

In [180]:
philips_a_info, pihips_b_info = e.get_positions().items()

In [181]:
print(philips_a_info)

('PHILIPS_A', 187)


In [183]:
philips_a_pos = philips_a_info[1]
print(philips_a_pos)

187


In [187]:
for s, p in e.get_positions().items():
    if p > 0:
        e.insert_order(s, price=1, volume=p, side='ask', order_type='ioc')
    elif p < 0:
        e.insert_order(s, price=100000, volume=-p, side='bid', order_type='ioc')  
print(e.get_positions())

{'PHILIPS_A': 0, 'PHILIPS_B': 0}
